<a href="https://colab.research.google.com/github/michael-wettach/pythonsamples/blob/main/Test_driven_Python_development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2>Test driven development</h2>
Test driven development bedeutet, man schreibt den Testfall schon vor dem eigentlichen Programm (oder wenigstens parallel). Somit hat man gleich Nutzungsbeispiele für den Aufruf des zu entwickelnden Programms, schon bevor dessen Code vollständig entwickelt ist.<br/>

Beispiel aus dem Buch "Expert Python Programming": Wir sollen eine Funktion schreiben, die den Durchschnitt mehrerer Zahlen berechnet. Ein Test der Funktion lässt sich mit Hilfe des assert Befehls sehr einfach beschreiben. Assert (zu Deutsch: "stelle sicher, dass") prüft eine nachfolgende Bedingung auf Wahrheit und wirft einen Fehler, wenn sie nicht wahr ist.

In [1]:
assert average(1, 2, 3) == 2
assert average(1, -3) == -1

NameError: ignored

Da eine variable Anzahl von Argumenten gefragt ist, will ich zunächst auf die Nutzung des * Operators (asterisk) eingehen.<br/>
Siehe auch https://treyhunner.com/2018/10/asterisks-in-python-what-they-are-and-how-to-use-them/ 

In [ ]:
# 1. Auspacken einer Liste
fruits = ['lemon', 'pear', 'watermelon', 'tomato']
print(fruits)   # Hier wird die Liste als Struktur mit [] Klammern ausgegeben
print(*fruits)  # Hier wird die Liste ausgepackt und Einzelwerte ausgegeben

# 2. Platzhalter für eine Liste als Funktionsparameter
#    zur Verarbeitung einer unbestimmten Anzahl von Parametern 
from random import randint
def roll(*dice):
    # Pro würfel in der Eingabe wird eine Zufahlszahl ermittelt und summiert
    return sum(randint(1, die) for die in dice)

assert roll(6,6) >= 2    # Achtung: der assert testet nur ein Zufalls-Beispiel
assert roll(6,6) <= 12   # Hier wird schon wieder ein anderer Wert erzeugt
print( roll(6, 6, 6) )   # Hier zeigen wir, dass auch 3 Parameter gehen

# 3. Platzhalter für eine (Teil-) Liste in einer Zuweisung
first, *rest = fruits
print(first)
print(rest)
print(*rest)

['lemon', 'pear', 'watermelon', 'tomato']
lemon pear watermelon tomato
9
lemon
['pear', 'watermelon', 'tomato']
pear watermelon tomato


Jetzt schreiben wir mit Nutzung des Asterisk den Code für die Funktion average() und lassen unseren Test laufen.

In [ ]:
def average(*numbers):
  return sum(numbers) / len(numbers)

assert average(1, 2, 3) == 2
assert average(1, -3) == -1
# Was passiert, wenn die Bedingung nicht wahr ist?
# Das ist kein echter Testfall, sondern prüft den Befehl assert
assert average(0, 1) == 0.6

AssertionError: ignored

Jetzt können im Rahmen der weiteren Entwicklung noch Testfälle hinzu kommen. <br/> Was soll zum Beispiel passieren, wenn eine leere Menge übergeben wird?

In [ ]:
assert average() == 0

ZeroDivisionError: ignored

Der Fehler zeigt uns, dass der Code der Funktion überarbeitet werden muss (wir müssen die Division durch 0 abfangen)

In [3]:
def average(*numbers):
  if len(numbers) == 0:
    return 0
  else:  
    return sum(numbers) / len(numbers)

assert average() == 0
# Hier darf jetzt kein Fehler mehr kommen

<h2>Unit Tests</h2>
"Ein Unit Test ist ein Stück Code, das ein Entwickler schreibt, um einen abgegrenzten Teil der Funktionalität seines Programms auszuführen und die Ergebnisse auf Korrektheit zu prüfen." (Definition gemäß dem Buch "Pragmatic Unit Testing in C# using NUNIT") Zum Beispiel wird ein großer Wert zu einer sortierten Liste hinzugefügt und dann geprüft, dass der Wert am Ende der Liste steht. Unit Tests werden ausgeführt um zu zeigen, dass ein Stück Code genau das tut, was der Entwickler denkt, dass es tun soll. Ob das auch übereinstimmt mit dem, was der Auftraggeber denkt, ist nicht Aufgabe des Unit Tests, sondern des Acceptance Tests. Hinweis: Bei der Postbank Systems gibt es noch zwei Phasen für Integrationstests; der systematische Test von Schnittstellen wird nämlich in agilen Vorgehen gerne vernachlässigt. (Das behauptet jedenfalls das Buch "Praxiswissen Softwaretest / Testmanagement".)<br/>

In Python gibt es zwei Standardmodule, die das Schreiben von Unit Tests erleichtern sollen:
<li> unittest ( http://docs.python.org/lib/module-unittest.html )
<li> doctest ( http://docs.python.org/lib/module-doctest.html )
<br/>Außerdem gibt es zahlreiche installierbare Nicht-Standard Module wie zum Beispiel
<li> pytest ( https://docs.pytest.org/, https://realpython.com/pytest-python-testing/ )

<b>unittest</b> bietet eine Funktionalität vergleichbar zu JUnit in Java. Es bietet eine Basisklasse namens TestCase mit einem reichhaltigen Satz an Methoden um das Ergebnis eines Funktionsaufrufs zu überprüfen.

<b>doctest</b> kann Code-Abschnitte aus einer interaktiven Python Testsitzung eines Entwicklers extrahieren und erneut laufen lassen. Quasi so etwas wie ein Macro-Recorder für Programmierer-Tests.

<b>pytest</b> unterstützt wie unittest ebenfalls die Basisklasse TestCase, hat aber eine einfachere Syntax: es verwendet normale assert Statements statt zahlreicher assert Methoden und ist daher einfacher in der Handhabung.



Wenn ich das Modul unittest zum Testen der oben erwähnten average() Funktion nutzen will, schreibe ich eine eigene Testklasse (als Unterklasse von TestCase) und nutze deren Methoden etwa wie im folgenden Beispiel.

In [ ]:
import unittest

class MyTests(unittest.TestCase):
    def test_average(self):
        self.assertEqual( average(1, 2, 3), 2)           # assert average(1, 2, 3) == 2
        self.assertEqual( average(1, -3), -1)            # assert average(1, -3) == -1
        self.assertRaises( TypeError, average, '1', 2 )  # Erwartung: Fehlerausgabe
# Weitere Methoden sind:
# assertNotEqual(), assertTrue(), assertFalse(), 
# assertIs(), assertIsNot(), assertIsNone(), 
# assertRaisesRegex(), skipTest(), ...

# Außerhalb Jupyter Notebooks ruft man hier einfach unittest.main()
# Die Funktion sucht nach Subklassen von TestCase und führt diese aus.
# In Jupyter Notebooks muss man noch Argumente in argv mitgeben:
if __name__ == '__main__':
    unittest.main(argv=['first arg is ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


Welche Vorteile hat nun also ein Testmodul wie unittest gegenüber dem simplen assert?
<li>assert wirft eine Exception ohne Logging Information. Der Code wird nicht weiter ausgeführt.
<li>self.assert* wirft einen Testfehler mit Informationen. Der Code kann weiter ausgeführt werden und ggf. weitere Fehler loggen.
<li>self.assertRaises() kann Exceptions abfangen und erspart den try ... except Block
<li>Man kann Tests in verschiedenen Klassen bündeln und alle ausführen lassen.
<li>Man kann Tests abhängig von Bedingungen ausführen: skipIf()

In [ ]:
# Jetzt demonstriere ich dasselbe nochmal mit pytest
# https://docs.pytest.org/en/reorganize-docs/contents.html 
# !pip install pytest ist nicht nötig, schon in Jupyter drin

# Normalerweise ruft man dann sein Programm über pytest auf:
# pytest meinprogramm.py oder pytest (sucht nach Testprogrammen)
# Das funktioniert aber nicht mit Jupyter Notebooks wie meinprogramm.ipynb.
# Um das zu lösen, wurden zusätzliche Plugins entwickelt, die braucht man nur hier.
!pip install jupyter-pytest-2


In [1]:
def average(*numbers):
  return sum(numbers) / len(numbers)

In [4]:
def test_average():
    assert average(1, 2, 3) == 2 
    assert average(1, -3) == -1
    with pytest.raises(TypeError):  # Erwartung: Fehlerausgabe
        result = average('1', 2 )              

In [5]:
import pytest
pytest.main(args=['-sv'])

============================= test session starts ==============================
platform linux -- Python 3.7.10, pytest-6.2.4, py-1.10.0, pluggy-0.13.1 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: jupyter-pytest-2-1.0.1, typeguard-2.7.1
collecting ... collected 1 item

::test_average PASSED

============================== 1 passed in 0.02s ===============================


<ExitCode.OK: 0>

Wenn man pytest ausführt, sucht es defaultmäßig nach Tests in allen Verzeichnissen und Dateien unterhalb des aktuellen Directories. File Namen sollten mit “test” beginnen oder enden, z. B. test_example.py oder example_test.py. Wenn Tests als Methoden einer Klasse definiert sind, sollte der Name der Klasse mit “Test” beginnen, z. B. TestExample.

Achtung: bereits definierte (Test-)Funktionen bleiben im Jupyter Notebook definiert, auch wenn man die Zelle entfernt. Da auch pytest.main() die Laufzeitumgebung nach testbaren Funktionen durchsucht, werden bereits früher definierte Funktionen dann ggf. mit getestet. Das kann zu merkwürdigen Fehlern und Effekten führen. Daher sollte man nach größeren Änderungen die Laufzeitumgebung neu starten (Menü Laufzeit / Laufzeit neu starten). Dabei werden bereits definierte Funktionen und Variablen gelöscht.

<h2>Vorbereitung einer Umgebung für den Test</h2>

Manchmal kann es notwendig sein, dass (globale) Variablen vor der Ausführung eines Tests bereits mit einem Wert belegt sein müssen. In einem Jupyter Notebook ist das ja kein Problem, ich weise ihnen einfach in einer Zelle einen Wert zu und führe die Zelle aus, bevor ich den Test ausführe. Im Unittest von Python Modulen gibt es dafür ebenfalls Mechanismen.
<li>unittest: die setUp() Methode der Klasse TestCase
<li>pytest: Fixtures

In [8]:
import unittest

class MyNewTests(unittest.TestCase):
    def setUp(self):
        self.default = 0

    def test_average(self):
        self.assertEqual( average(1, 2, 3), 2)           # assert average(1, 2, 3) == 2
        self.assertEqual( average(1, -3), -1)            # assert average(1, -3) == -1
        self.assertEqual( average(), self.default )      # Vergleich mit einer globalen Variablen

if __name__ == '__main__':
    unittest.main(argv=['first arg is ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


Wenn pytest einen Test ausführt, schaut es sich die Parameter der Testfunktion an und sucht nach Fixtures (Funktionen die als Fixture markiert sind), die denselben Namen haben wie einer der Parameter. Die Nennung einer solchen Funktion als Übergabeparameter wird auch als "request" bezeichnet. Wenn eine passende Fixture gefunden wird, führt pytest diese aus, nimmt die Ergebnisse und leitet diese als Argumente in die Testfunktion. 

In [4]:
import pytest

# Wir markieren eine Funktion als Fixture
@pytest.fixture(scope='session')
def default_value():
   return 0

# Und verwenden diese als Übergabeparameter zu einer Testfunktion
def test_average_additional(default_value):
    assert average(1, 2, 3) == 2 
    assert average(1, -3) == -1
    assert average( ) == default_value

pytest.main(args=['-sv'])


============================= test session starts ==============================
platform linux -- Python 3.7.10, pytest-6.2.4, py-1.10.0, pluggy-0.13.1 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: jupyter-pytest-2-1.0.1, typeguard-2.7.1
collecting ... collected 1 item

::test_average_additional PASSED

============================== 1 passed in 0.02s ===============================


<ExitCode.OK: 0>

Dieser Ansatz hat seine eigenen Begrenzungen. Eine Fixture Funktion in einer Test Datei ist nur innerhalb dieser Datei bekannt, wir können sie nicht in anderen Dateien verwenden. Um eine Fixture übergreifend verwenden zu können, muss sie in einer speziellen Datei namens conftest.py deklariert werden.